In [1]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import java.io.File
spark.stop()

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002716.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1620827369570)
SparkSession available as 'spark'


import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import java.io.File


In [2]:
val spark = SparkSession
 .builder
 .appName("Goodreads-Books")
 .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6796e051


**Funciones**

In [4]:
// Función para comprobar que existe la columna, ya que hay files con menos columnas
def hasColumn(df: DataFrame, path: String) = {
  if (Try(df(path)).isSuccess){
    df(path)
  }else{
    lit(null)
  }
}

// Recojo el nombre de los archivos .csv de los libros
def getListOfFiles(dir: String): Array[String] = {
  val d = new File(dir)
    if (d.exists && d.isDirectory) {
      d.listFiles.filter(_.isFile)
        .filter(_.getName.contains("book"))
        .map(_.getPath).toArray
    } else {
      Array[String]()
    }
}

hasColumn: (df: org.apache.spark.sql.DataFrame, path: String)org.apache.spark.sql.Column
getListOfFiles: (dir: String)Array[String]


In [5]:
val ruta = "C:/Users/mario.serrano/Desktop/Goodreads-Books/datasets"
val bookFiles = getListOfFiles(ruta)

ruta: String = C:/Users/mario.serrano/Desktop/Goodreads-Books/datasets
bookFiles: Array[String] = Array(C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1-100k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1000k-1100k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book100k-200k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1100k-1200k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1200k-1300k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1300k-1400k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1400k-1500k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1500k-1600k.csv, C:\Users\mario.serrano\Desktop\Goodreads-Books\datasets\book1600k-1700k.csv, C:\Use...


In [33]:
val dfList = bookFiles.map{ file => spark.read.option("header","true")
                           .option("quote", "\"")
                           .option("escape", "\"")
                           .option("ignoreLeadingWhiteSpace","true")
                           .option("ignoreTrailingWhiteSpace","true")
                           //.option("multiLine","true")  // Hay saltos de líneas en algunos registros, falla en jupyter duplicate
                           .csv(file)}  

dfList: Array[org.apache.spark.sql.DataFrame] = Array([Id: string, Name: string ... 16 more fields], [Id: string, Name: string ... 18 more fields], [pagesNumber: string, Authors: string ... 16 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Authors: string, CountsOfReview: string ... 17 more fields], [Id: string, Name: string ... 17 more fields], [Id: string, Name: string ... 17 more fields], [Id: string, Name: string ... 17 more fields], [Publisher: string, RatingDistTotal: string ... 16 more fields], [Id: string, Name: string ... 17 more fi...


**A continuación voy a ordenar todos los dataframes, y utilizo la función creada anteriormente <br/>
para rellenar con null si no existe esa columna**

In [7]:
val dfList2 = dfList.map{df =>  df.withColumn("Description", hasColumn(df, "Description"))
                                  .withColumn("Count of text reviews",hasColumn(df,"Count of text reviews"))}

dfList2: Array[org.apache.spark.sql.DataFrame] = Array([Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [pagesNumber: string, Authors: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Authors: string, CountsOfReview: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Id: string, Name: string ... 18 more fields], [Publisher: string, RatingDistTotal: string ... 18 more fields], [Id: string, Name: string ... 18 more f...


**Una vez ordenado realizo la union de los dataframes, utilizo el distinct ya que tenemos unos cuantos datos duplicados**

In [20]:
val unionDF = dfList2.reduce(_ unionByName _).coalesce(5)

unionDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Id: string, Name: string ... 18 more fields]


**Doy formato y limpio datos de las columnas RatingDist5,4,3,2,1 <br/>
También me doy cuenta que el PublishMonth y el PublishDay están cambiados ya que los valores de PublishDay van de 1 a 12**

In [22]:
val df = unionDF.withColumn("Id",$"Id".cast(IntegerType))
  .withColumn("Rating",$"Rating".cast(DoubleType))
  .withColumn("PublishYear",$"PublishYear".cast(IntegerType))
  .withColumn("PublishMonth",$"PublishMonth".cast(IntegerType))
  .withColumn("PublishDay",$"PublishDay".cast(IntegerType))
  .withColumn("RatingDist5",regexp_extract($"RatingDist5","""5:(\d+)""",1).cast(IntegerType))
  .withColumn("RatingDist4",regexp_extract($"RatingDist4","""4:(\d+)""",1).cast(IntegerType))
  .withColumn("RatingDist3",regexp_extract($"RatingDist3","""3:(\d+)""",1).cast(IntegerType))
  .withColumn("RatingDist2",regexp_extract($"RatingDist2","""2:(\d+)""",1).cast(IntegerType))
  .withColumn("RatingDist1",regexp_extract($"RatingDist1","""1:(\d+)""",1).cast(IntegerType))
  .withColumn("RatingDistTotal",regexp_extract($"RatingDistTotal","""total:(\d+)""",1).cast(IntegerType))
  .withColumn("CountsOfReview",$"CountsOfReview".cast(IntegerType))
  .withColumn("pagesNumber",$"pagesNumber".cast(IntegerType))
  .withColumn("Count of text reviews",$"Count of text reviews".cast(IntegerType))
  .withColumnRenamed("Count of text reviews","CountOfTextReviews")
  .withColumnRenamed("PublishMonth","PublishDay1")
  .withColumnRenamed("PublishDay","PublishMonth")
  .withColumnRenamed("PublishDay1","PublishDay")

df: org.apache.spark.sql.DataFrame = [Id: int, Name: string ... 18 more fields]


In [23]:
df.printSchema

root
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- RatingDist1: integer (nullable = true)
 |-- pagesNumber: integer (nullable = true)
 |-- RatingDist4: integer (nullable = true)
 |-- RatingDistTotal: integer (nullable = true)
 |-- PublishDay: integer (nullable = true)
 |-- PublishMonth: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- CountsOfReview: integer (nullable = true)
 |-- PublishYear: integer (nullable = true)
 |-- Language: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- RatingDist2: integer (nullable = true)
 |-- RatingDist5: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- RatingDist3: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- CountOfTextReviews: integer (nullable = true)



**Cambio nulls y redondeo el Rating a 2 decimales**

In [24]:
val cleanDF = df.withColumn("Language",when($"Language".isNotNull, $"Language").otherwise("n/a"))
  .withColumn("ISBN",when($"ISBN".isNotNull, $"ISBN").otherwise("n/a"))
  .withColumn("Description",when($"Description".isNotNull, $"Description").otherwise("Description not available"))
  .withColumn("CountOfTextReviews",when($"CountOfTextReviews".isNotNull, $"CountOfTextReviews").otherwise(0))
  .withColumn("Rating",round($"Rating",2))

cleanDF: org.apache.spark.sql.DataFrame = [Id: int, Name: string ... 18 more fields]


**Leo los otros archivos realacionados con los ratings dados por usuarios**

In [25]:
val userRDF = spark.read.format("csv")
  .option("header","true")
  .option("quote", "\"")
  .option("escape", "\"")
  .load("C:/Users/mario.serrano/Desktop/Goodreads-Books/datasets/user*.csv")
userRDF.printSchema
userRDF.cache()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Rating: string (nullable = true)



userRDF: org.apache.spark.sql.DataFrame = [ID: string, Name: string ... 1 more field]
res11: userRDF.type = [ID: string, Name: string ... 1 more field]


**Me he creado un dataframe pivoteando los valores de los ratings por usuarios y limpiado los nulls, <br/>
se podría utilizar para ver si tiene un buen rating o no un libro**

In [26]:
val pivotDF = userRDF.groupBy("Name")
  .pivot("Rating")
  .count()
val booksRatingDF = pivotDF.select("Name","it was amazing","really liked it","liked it","it was ok","did not like it")
  .withColumn("it was amazing",when($"it was amazing".isNull, 0).otherwise($"it was amazing"))
  .withColumn("really liked it",when($"really liked it".isNull, 0).otherwise($"really liked it"))
  .withColumn("liked it",when($"liked it".isNull, 0).otherwise($"liked it"))
  .withColumn("it was ok",when($"it was ok".isNull, 0).otherwise($"it was ok"))
  .withColumn("did not like it",when($"did not like it".isNull, 0).otherwise($"did not like it"))
booksRatingDF.show(10, false)

+--------------------------------------------------+--------------+---------------+--------+---------+---------------+
|Name                                              |it was amazing|really liked it|liked it|it was ok|did not like it|
+--------------------------------------------------+--------------+---------------+--------+---------+---------------+
|Fun Home: A Family Tragicomic                     |33            |32             |7       |0        |0              |
|A Blind Man Can See How Much I Love You           |3             |0              |2       |1        |0              |
|Assembling My Father: A Daughter's Detective Story|0             |0              |1       |0        |0              |
|The Night Watch                                   |2             |3              |6       |1        |0              |
|Weight: The Myth of Atlas and Heracles            |2             |7              |3       |2        |0              |
|Art and Lies                                   

pivotDF: org.apache.spark.sql.DataFrame = [Name: string, This user doesn't have any rating: bigint ... 5 more fields]
booksRatingDF: org.apache.spark.sql.DataFrame = [Name: string, it was amazing: bigint ... 4 more fields]


**Lo guardo en parquet para realizar las consultas**

In [27]:
cleanDF.write.format("parquet").mode("overwrite").save("C:/Users/mario.serrano/Desktop/Goodreads-Books/datasets/parquet")

In [28]:
val columnarBooksDF = spark.read.format("parquet").load("C:/Users/mario.serrano/Desktop/Goodreads-Books/datasets/parquet/*.parquet")
columnarBooksDF.printSchema

root
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- RatingDist1: integer (nullable = true)
 |-- pagesNumber: integer (nullable = true)
 |-- RatingDist4: integer (nullable = true)
 |-- RatingDistTotal: integer (nullable = true)
 |-- PublishDay: integer (nullable = true)
 |-- PublishMonth: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- CountsOfReview: integer (nullable = true)
 |-- PublishYear: integer (nullable = true)
 |-- Language: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- RatingDist2: integer (nullable = true)
 |-- RatingDist5: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- RatingDist3: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- CountOfTextReviews: integer (nullable = true)



columnarBooksDF: org.apache.spark.sql.DataFrame = [Id: int, Name: string ... 18 more fields]


**1.- Rating promedio de todos los libros**

In [29]:
columnarBooksDF.select(round(avg("Rating"),2).as("Average Rating")).show(5)

+--------------+
|Average Rating|
+--------------+
|       3761.15|
+--------------+



**2.- Rating promedio de los libros por autor**

In [35]:
columnarBooksDF.groupBy("Authors")
  .agg(round(avg("Rating"),1).as("Average Rating"))
  .where(col("Average Rating").isNotNull)
  .show()

+--------------------+--------------+
|             Authors|Average Rating|
+--------------------+--------------+
|       Ruthie Bolton|           4.1|
|      Richard Taylor|           3.7|
|    M. Brinton Lykes|           4.0|
|          Carl Jones|           2.5|
|       Jim Hightower|           3.9|
|       Bob Flowerdew|           3.0|
|         Maria Twist|           0.0|
|       Triumph Books|           2.8|
|Muhammad Ibn Jari...|           4.4|
| M. Basil Pennington|           3.7|
|        Jackie Marsh|           3.3|
|        Tom Tumbusch|           4.4|
|     Achmed Abdullah|           0.8|
|      Nina Berberova|           3.5|
|      Michael Tanner|           2.7|
|        Fred Ehrlich|           3.4|
|      Elizabeth Reis|           3.8|
|      John C. Pickup|           2.5|
|     Stephen McGinty|           3.2|
|   Thomas Zimmerling|           0.0|
+--------------------+--------------+
only showing top 20 rows



**3.- Rating promedio de los libros por Publisher**

In [31]:
columnarBooksDF.groupBy("Publisher")
  .agg(round(avg("Rating"),2).as("Average Rating"))
  .show()

+--------------------+--------------+
|           Publisher|Average Rating|
+--------------------+--------------+
|               Seuil|          3.36|
|               Wiley|          2.54|
|John Benjamins Pu...|          1.53|
|Rourke Educationa...|           2.4|
|       Triumph Books|          3.58|
|          BradyGames|          3.18|
|Louisiana State U...|          3.21|
|  Coffee House Press|          3.83|
|     Putnam Juvenile|          3.69|
| Stride Publications|          1.72|
|        Tantor Media|          3.78|
|    George Braziller|          3.55|
|             Lorimer|          2.29|
|American Heritage...|          2.59|
|        Alfred Music|          1.94|
|   B.E.S. Publishing|          3.42|
|      Véhicule Press|          2.58|
|Bristol Classical...|          3.48|
|University Press ...|          1.35|
|William B. Eerdma...|          3.49|
+--------------------+--------------+
only showing top 20 rows



**4.- Número promedio de páginas de todos los libros**

In [34]:
columnarBooksDF.select(round(avg("pagesNumber"),2).as("Average Pages")).show()

+-------------+
|Average Pages|
+-------------+
|       534.44|
+-------------+



**5.- Número promedio de páginas de todos los libros por autor**

In [41]:
columnarBooksDF.groupBy("Authors")
  .agg(round(avg("pagesNumber")).cast(IntegerType).as("Average Pages"))
  .where(col("Average Pages").isNotNull)
  .show()

+--------------------+-------------+
|             Authors|Average Pages|
+--------------------+-------------+
|       Ruthie Bolton|          300|
|      Richard Taylor|          239|
|    M. Brinton Lykes|          392|
|          Carl Jones|          192|
|       Jim Hightower|          315|
|       Bob Flowerdew|          200|
|         Maria Twist|          128|
|       Triumph Books|          174|
|Muhammad Ibn Jari...|          300|
| M. Basil Pennington|          168|
|        Jackie Marsh|          282|
|        Tom Tumbusch|          156|
|     Achmed Abdullah|          288|
|      Nina Berberova|          271|
|      Michael Tanner|          237|
|        Fred Ehrlich|           33|
|      Elizabeth Reis|          293|
|      John C. Pickup|          867|
|     Stephen McGinty|          245|
|   Thomas Zimmerling|           24|
+--------------------+-------------+
only showing top 20 rows



**6.- Número promedio de páginas de todos los libros por Publisher**

In [43]:
columnarBooksDF.groupBy("Publisher")
  .agg(round(avg("pagesNumber")).cast(IntegerType).as("Average Pages"))
  .show()

+--------------------+-------------+
|           Publisher|Average Pages|
+--------------------+-------------+
|               Seuil|          317|
|               Wiley|          400|
|John Benjamins Pu...|          325|
|Rourke Educationa...|           30|
|       Triumph Books|          214|
|          BradyGames|          180|
|Louisiana State U...|          249|
|  Coffee House Press|          187|
|     Putnam Juvenile|           69|
| Stride Publications|          114|
|        Tantor Media|            9|
|    George Braziller|          200|
|             Lorimer|          171|
|American Heritage...|          262|
|        Alfred Music|           80|
|   B.E.S. Publishing|          105|
|      Véhicule Press|          178|
|Bristol Classical...|          203|
|University Press ...|          243|
|William B. Eerdma...|          291|
+--------------------+-------------+
only showing top 20 rows



**7.- Número promedio de libros publicados por autor**

In [45]:
columnarBooksDF.groupBy("Authors")
  .count()
  .agg(round(avg("count"),2).as("Average Books"))
  .show()

+-------------+
|Average Books|
+-------------+
|         2.75|
+-------------+



**8.- Ordenar los libros de mayor a menor (Top 15) por número de ratings dados por usuarios (excluir aquellos valores sin rating)**

In [46]:
userRDF.groupBy("Name")
  .count()
  .where($"Name" =!= "Rating")
  .select($"Name",$"count".as("Ratings Received"))
  .orderBy(desc("count"))
  .show(false)

+------------------------------------------------------------+----------------+
|Name                                                        |Ratings Received|
+------------------------------------------------------------+----------------+
|The Catcher in the Rye                                      |985             |
|The Great Gatsby                                            |885             |
|The Da Vinci Code (Robert Langdon, #2)                      |846             |
|To Kill a Mockingbird                                       |830             |
|1984                                                        |756             |
|The Kite Runner                                             |749             |
|Harry Potter and the Sorcerer's Stone (Harry Potter, #1)    |728             |
|Animal Farm                                                 |717             |
|Harry Potter and the Goblet of Fire (Harry Potter, #4)      |639             |
|Harry Potter and the Prisoner of Azkaba

**9.- Obtener Top 5 de ratings más frecuentes otorgados por usuarios**

In [47]:
userRDF.groupBy("Rating")
  .count()
  .orderBy(desc("count"))
  .limit(5)
  .show()

+---------------+------+
|         Rating| count|
+---------------+------+
|really liked it|132808|
|       liked it| 96047|
| it was amazing| 92354|
|      it was ok| 28811|
|did not like it|  7811|
+---------------+------+

